[![AnalyticsDojo](https://github.com/rpi-techfundamentals/fall2018-materials/blob/master/fig/final-logo.png?raw=1)](http://rpi.analyticsdojo.com)
<center><h1>Pytorch - Revisiting Titanic</h1></center>
<center><h3><a href = 'http://rpi.analyticsdojo.com'>rpi.analyticsdojo.com</a></h3></center>


In [1]:
!pip install torch torchvision

In [2]:
import numpy as np
import pandas as pd
import pandas as pd
from sklearn import preprocessing

train= pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')



In [3]:
#Print to standard output, and see the results in the "log" section below after running your script
train.head()

,AnimalID,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
0,A671945,Hambone,2014-02-12 18:22:00,Return_to_owner,NaN,Dog,Neutered Male,1 year,Shetland Sheepdog Mix,Brown/White
1,A656520,Emily,2013-10-13 12:44:00,Euthanasia,Suffering,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Cream Tabby
2,A686464,Pearce,2015-01-31 12:28:00,Adoption,Foster,Dog,Neutered Male,2 years,Pit Bull Mix,Blue/White
3,A683430,NaN,2014-07-11 19:09:00,Transfer,Partner,Cat,Intact Male,3 weeks,Domestic Shorthair Mix,Blue Cream
4,A667013,NaN,2013-11-15 12:52:00,Transfer,Partner,Dog,Neutered Male,2 years,Lhasa Apso/Miniature Poodle,Tan


In [4]:
#This replaces the Null Names with the variable "Nameless"
train.loc[train.Name.isnull(), "Name"]='Unnamed'
test.loc[test.Name.isnull(), "Name"]='Unnamed'

In [5]:
#Evaluate SexuponOutcome
train.loc[train.SexuponOutcome.isnull(), "SexuponOutcome"]='Unknown'
test.loc[test.SexuponOutcome.isnull(), "SexuponOutcome"]='Unknown'

In [6]:
#Function that converts all AgeuponOutcome to a consistent unit (ie. years) and removes entires with null Ages
train.loc[train.AgeuponOutcome.isnull(), 'AgeuponOutcome'] = '0 days'
test.loc[test.AgeuponOutcome.isnull(), 'AgeuponOutcome'] = '0 days'
def calc_age(df):
    #print(type(df['AgeuponOutcome']))
    ages = df['AgeuponOutcome'].apply(lambda x: str(x).split(' '))
    df['AgeinDays'] = 0
    for i in range(len(df)):
        if ages[i][1] == 'years':
            #print(int(ages[i][0]))
            df.loc[i, 'AgeinDays'] = int(ages[i][0])*365
        elif ages[i][1] == 'years':
            df.loc[i, 'AgeinDays'] = int(ages[i][0])*365
        elif ages[i][1] == 'weeks':
            #print(int(ages[i][0])*7)
            df.loc[i, 'AgeinDays'] = int(ages[i][0])*7
        elif ages[i][1]== 'week':
            df.loc[i, 'AgeinDays'] = int(ages[i][0])*7
        elif ages[i][1] == 'months':
            #print(int(ages[i][0])*12)
            df.loc[i, 'AgeinDays'] = int(ages[i][0])*12
        elif ages[i][1] == 'month':
            df.loc[i, 'AgeinDays'] = int(ages[i][0])*12
        elif ages[i][1] == 'years':
            #print(int(ages[i][0])*365)
            df.loc[i, 'AgeinDays'] = int(ages[i][0])*365
        elif ages[i][1] == 'year':
            df.loc[i, 'AgeinDays'] = int(ages[i][0])*365

calc_age(train)
calc_age(test)

In [7]:
import datetime
def transform_date(df):
    date = pd.to_datetime(df.DateTime)
    df['Hour'] = date.dt.hour
    df['WeekDay'] = date.dt.weekday
    df['Month'] = date.dt.month
    df['Year'] = date.dt.year

transform_date(train)
transform_date(test)

In [8]:
def category_name(df):
    for i in range(len(df)):
        if df['Name'][i] == 'Unnamed':
            df.loc[i, 'NameCategory'] = 0
        else:
            df.loc[i, 'NameCategory'] = 1
category_name(train)
category_name(test)

In [9]:
def category_age(df):
    for i in range(len(df)):
        if df['AgeinDays'][i] <= 365:
            df.loc[i, 'AgeCategory'] = 'Baby'
        elif 365< df['AgeinDays'][i] <= 1095:
            df.loc[i, 'AgeCategory'] = 'Young adult'
        elif 1095 < df['AgeinDays'][i] < 1825:
            df.loc[i, 'AgeCategory'] = 'Adult'
        else:
            df.loc[i, 'AgeCategory'] = 'Senior'
category_age(train)
category_age(test)

In [10]:
def category_breed(df):
    df['IsMix'] = df['Breed'].apply(lambda x: 1 if 'Mix' in x else 0)
    breeds = train['Breed'].apply(lambda x: x.split('/'))
    df['Breed1'] = breeds.apply(lambda x: x[0].replace(' Mix', ''))
    df['Breed2'] = breeds.apply(lambda x: x[1] if len(x)>1 else x[0].replace(' Mix', ''))
    df['MultipleBreeds'] = df['Breed'].apply(lambda x: 1 if '/' in x else 0)
category_breed(train)
category_breed(test)

In [11]:
#Used to split 
def category_color(df):
    color1 = ""
    color2 = ""
    colors = df['Color'].apply(lambda x: x.split('/'))
    for i in range(len(colors)):
        if len(colors[i]) == 1:
            hit = False
            for j in range(len(colors[i])):
                for k in range(len(colors[i][j])):
                    if colors[i][j][k] == " ":
                        hit = True
                        color_split_set = colors[i][j].split()
                        df.loc[i,'Color1'] = color_split_set[0]
                        df.loc[i,'Color2'] = color_split_set[0]
                if hit == False:
                    df.loc[i,'Color1'] = colors[i][j]
                    df.loc[i,'Color2'] = colors[i][j]
        else:
            hit1 = False
            hit2 = False

            for j in range(len(colors[i])):
                if j == 0:
                    for k in range(len(colors[i][j])):
                        if colors[i][j][k] == " ":
                            hit1 = True
                            color_split_set = colors[i][j].split()
                            df.loc[i,'Color1'] = color_split_set[0]
                        
                    if hit1 == False:
                        df.loc[i,'Color1'] = colors[i][j]
                        
                else:
                    for k in range(len(colors[i][j])):
                        if colors[i][j][k] == " ":
                            hit2 = True
                            color_split_set = colors[i][j].split()
                            df.loc[i,'Color2'] = color_split_set[0]
                    if hit2 == False:
                        df.loc[i,'Color2'] = colors[i][j]               
category_color(train)
category_color(test)

In [16]:
#Drop some colums that won't be modeled. 
train_min = train.drop(['AnimalID', 'Name', 'DateTime', 'OutcomeSubtype', 'AgeuponOutcome', 'Breed', 'Color'], axis = 1)
test_min= test.drop(['ID', 'Name', 'DateTime', 'AgeuponOutcome', 'Breed', 'Color'], axis = 1)
test_min.head()

,AnimalType,SexuponOutcome,AgeinDays,Hour,WeekDay,Month,Year,NameCategory,AgeCategory,IsMix,Breed1,Breed2,MultipleBreeds,Color1,Color2
0,Dog,Intact Female,120,12,0,10,2015,1.0,Baby,1,Shetland Sheepdog,Shetland Sheepdog,0,Red,White
1,Dog,Spayed Female,730,17,5,7,2014,1.0,Young adult,0,Domestic Shorthair,Domestic Shorthair,1,Black,Tan
2,Cat,Neutered Male,365,12,2,1,2016,1.0,Baby,1,Pit Bull,Pit Bull,0,Brown,Brown
3,Dog,Intact Male,48,18,5,12,2013,1.0,Baby,1,Domestic Shorthair,Domestic Shorthair,0,Tricolor,Tricolor
4,Dog,Neutered Male,730,17,3,9,2015,1.0,Young adult,1,Lhasa Apso,Miniature Poodle,0,White,White


In [19]:
#Adding .values will make whatever comes out into a numpy array. 
X_train = train_min.iloc[:, 1:15].values
Y_train = train_min.iloc[:, 0].values
X_test = test_min.iloc[:, 0:15].values

print(X_test[1], X_train[1])
print(len(X_test[1]), len(X_train[1]))

['Dog' 'Spayed Female' 730 17 5 7 2014 1.0 'Young adult' 0
 'Domestic Shorthair' 'Domestic Shorthair' 1 'Black' 'Tan'] ['Cat' 'Spayed Female' 365 12 6 10 2013 1.0 'Baby' 1 'Domestic Shorthair'
 'Domestic Shorthair' 0 'Cream']
15 14


In [90]:
# Previously we used get dummies (part of pandas)
#Here we will first transform string labels to numeric categories. 
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X_train[:, 0] = labelencoder_X_1.fit_transform(X_train[:, 0]) #AnimalType
X_train[:, 1] = labelencoder_X_1.fit_transform(X_train[:, 1]) #Sex
X_train[:, 8] = labelencoder_X_1.fit_transform(X_train[:, 8]) #AgeCategory
X_train[:, 10] = labelencoder_X_1.fit_transform(X_train[:, 10]) #Breed1
X_train[:, 11] = labelencoder_X_1.fit_transform(X_train[:, 11]) #Breed2
X_train[:, 13] = labelencoder_X_1.fit_transform(X_train[:, 13]) #Color1

y_train[:, 0] = labelencoder_X_1.fit_transform(y_train[:])

labelencoder_X_2 = LabelEncoder()
X_test[:, 0] = labelencoder_X_2.fit_transform(X_test[:, 0])
X_test[:, 1] = labelencoder_X_2.fit_transform(X_test[:, 1])
X_test[:, 8] = labelencoder_X_1.fit_transform(X_test[:, 8]) 
X_test[:, 10] = labelencoder_X_2.fit_transform(X_test[:, 10])
X_test[:, 11] = labelencoder_X_2.fit_transform(X_test[:, 11])
X_test[:, 13] = labelencoder_X_2.fit_transform(X_test[:, 13])
X_test[:, 14] = labelencoder_X_1.fit_transform(X_test[:, 14])
X_test.shape

ValueError: bad input shape (21383, 5)

In [49]:
print(pd.DataFrame(X_test).isna().sum())
print(pd.DataFrame(X_test).isna().sum())

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    0
27    0
28    0
29    0
30    0
31    0
32    0
33    0
34    0
35    0
36    0
37    0
38    0
39    0
40    0
41    0
42    0
dtype: int64
0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    0
27    0
28    0
29    0
30    0
31    0
32    0
33    0
34    0
35    0
36    0
37    0
38    0
39    0
40    0
41    0
42    0
dtype: int64


In [85]:
# Converting numeric categories values to one-hot representation
one_hot_encoder_train = OneHotEncoder(categorical_features = [0, 1, 8, 10, 11, 13])
one_hot_encoder_y_train = OneHotEncoder(categorical_features = [0])
one_hot_encoder_test = OneHotEncoder(categorical_features = [0, 1, 8, 10, 11, 13, 14])
X_train = one_hot_encoder.fit_transform(X_train).toarray()
y_train = pd.get_dummies(y_train[:])
X_test = one_hot_encoder.fit_transform(X_test).toarray()

In [74]:
#Double check no missing values or values that aren't numeric.
print(np.isnan(X_train).sum(),np.isnan(X_test).sum())
print(X_train[0])

0 0
[  0.   1.   1.   0.   0.   1.   1.   0.   0.   1.   1.   0.   0.   1.
   0.   1.   1.   0.   0.   1.   0.   1.   0.   1.   0.   1.   0.   1.
   0.   1.   1.   0.   1.   0.   1.   1.   0.   0.   1.   1.   0.   1.
   0.   1.   1.   0.   0.   1.   0.   0.   0.   0.   1.   1.   0.   1.
   0.   0.   1.   0.   1.   0.   0.   1.   0.   0.   0.   0.   1.   0.
   0.   0.   0.   1.   1. 365.  18.   2.   1.   1. 181. 181.   0.   4.]


In [82]:
#Split the data
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size = 0.2)
x_train.shape

(21383,)

In [76]:
y_val.shape

(5346,)

In [77]:
#Define the model 
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        #Note that 17 is the number of columns in the input matrix. 
        self.fc1 = nn.Linear(18, 270)
        #270 is arbitrary, but needs to be consistent.  2 is the number of classes in the output (died/survived)
        self.fc2 = nn.Linear(270, 2)
        
    def forward(self, x):
        x = self.fc1(x)
        x = F.dropout(x, p=0.1)
        x = F.relu(x)
        x = self.fc2(x)
        x = torch.sigmoid(x)  #signmoid flattens to probability 0-1. 
        
        return x
    
net = Net()

In [78]:
#Define training
batch_size = 50
num_epochs = 100
learning_rate = 0.01
batch_no = len(x_train) // batch_size

In [79]:
#define loss function and optimizer 
#Adam is a specific flavor of gradient decent which is typically better
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

In [83]:
print(y_train)

['Adoption' 'Adoption' 'Transfer' ... 'Return_to_owner' 'Adoption'
 'Transfer']


In [81]:
from sklearn.utils import shuffle
from torch.autograd import Variable
running_loss = 0.0
for epoch in range(num_epochs):
    x_train, y_train = shuffle(x_train, y_train)
    # Mini batch learning
    for i in range(batch_no):
        start = i * batch_size
        end = start + batch_size
        inputs = Variable(torch.FloatTensor(x_train[start:end]))
        labels = Variable(torch.LongTensor(y_train[start:end]))
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        
    print('Epoch {}'.format(epoch+1), "loss: ",running_loss)
    running_loss = 0.0

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: double, float, float16, int64, int32, and uint8.

In [45]:
#This is a little bit tricky to get the resulting prediction.  
def calculate_accuracy(x,y=[]):
  # Evaluate the model with the test set. 
  test_var = Variable(torch.FloatTensor(x), requires_grad=True)
  with torch.no_grad():   
      result = net(test_var) #This outputs the probability for each class.
  values, labels = torch.max(result, 1)
  if len(y) != 0:
      num_right = np.sum(labels.data.numpy() == y)
      print('Accuracy {:.2f}'.format(num_right / len(y)), "for a total of ", len(y), "records")
  else:
      print("returning predictions")
      return labels.data.numpy()
 
  


In [46]:
# Check out the accuracy. 
calculate_accuracy(x_train, y_train)
calculate_accuracy(x_val, y_val)
predictions=calculate_accuracy(X_test)
len(predictions)
predictions

RuntimeError: size mismatch, m1: [21383 x 42], m2: [18 x 270] at /pytorch/aten/src/TH/generic/THTensorMath.cpp:940

In [ ]:
'''#Writing to File
submission=pd.DataFrame(test.loc[:,['PassengerId']])
submission['Survived']=predictions
#Any files you save will be available in the output tab below

submission.to_csv('submission.csv', index=False)
'''

 

In [ ]:
#from google.colab import files
#files.download('submission.csv')

In [ ]:
# Evaluate the model
test_var = Variable(torch.FloatTensor(x_val), requires_grad=True)
with torch.no_grad():
    result = net(test_var)
values, labels = torch.max(result, 1)
num_right = np.sum(labels.data.numpy() == y_val)
print('Accuracy {:.2f}'.format(num_right / len(y_val)))